In [5]:
import pandas as pd
import numpy as np
import joblib

import pathlib

In [6]:
data_path = pathlib.Path("../data")
train = pd.read_csv(data_path/'Train.csv')
train_extra = pd.read_csv(data_path/'train_add.csv')
train_extra_2 = pd.read_csv(data_path/'train_add_20201029.csv')

label2code = {word: idx for idx, word in enumerate(train.label.unique().tolist())}
code2label = {v:k for k,v in label2code.items()}

train_df = pd.concat([train, train_extra, train_extra_2], axis=0).reset_index(drop=True)

In [13]:
train_class_dict = train_df.label.value_counts() / train_df.shape[0]

In [14]:
subm = pd.read_csv("submissions/gm_w_new_dens_2.csv")
print(subm.shape)

(1017, 194)


In [15]:
speech_prod_dict = joblib.load("probs_dict.joblib")

In [16]:
subm_pred_cols = subm.drop("fn", axis=1).columns.values
replace_preds = np.array([train_class_dict[i] for i in subm_pred_cols])

array(['audio_files/00118N3.wav', 'audio_files/00P0NMV.wav',
       'audio_files/01QEEZI.wav', ..., 'audio_files/ZSC69U3.wav',
       'audio_files/ZU1I8MR.wav', 'audio_files/ZYO56VD.wav'], dtype=object)

In [33]:
updated_preds = list()
thr = 0.01
for fn, cur_preds in zip(subm.fn.values, subm[subm_pred_cols].values):
    fn_probs = speech_prod_dict[fn.split("/")[-1]]
    mean_fn_probs = np.mean(fn_probs)
    if mean_fn_probs >= thr:
        updated_preds.append(cur_preds)
    else:
        updated_preds.append(replace_preds)
        #updated_preds.append(np.ones(n_classes) / n_classes)
        print(f"updated weights for {fn} with mean speech probs {mean_fn_probs}")
        print('------')
updated_preds = np.array(updated_preds)
assert updated_preds.shape == subm[subm_pred_cols].values.shape

updated weights for audio_files/05KISR5.wav with mean speech probs 0.003523151212538354
------
updated weights for audio_files/37839CJ.wav with mean speech probs 0.004766196737983736
------
updated weights for audio_files/DWLVCY9.wav with mean speech probs 0.0028154577268062583
------
updated weights for audio_files/HJ9ACW2.wav with mean speech probs 0.007202305982861182
------
updated weights for audio_files/J0N19SG.wav with mean speech probs 0.0032605875760388834
------
updated weights for audio_files/JFS7FAK.wav with mean speech probs 0.003661880148169787
------
updated weights for audio_files/LZL66TN.wav with mean speech probs 0.003789361209748088
------
updated weights for audio_files/OFFQD80.wav with mean speech probs 0.007900190950204857
------
updated weights for audio_files/RI11IKN.wav with mean speech probs 0.0038822527529428835
------
updated weights for audio_files/SY0BKC1.wav with mean speech probs 0.007236266022791033
------


In [50]:
new_subm = subm.copy()

In [51]:
new_subm[subm_pred_cols] = updated_preds

In [52]:
np.mean(new_subm[subm_pred_cols].values == subm[subm_pred_cols].values)

0.9872173058013766

In [37]:
new_subm.to_csv("submissions/gm_w_new_dens_2_PP_1.csv", index=False)